# Spike sorting w/o drift corr

10 min recording | +/- drift correction | high-pass filt.| - bad channels

author: laquitainesteeve@gmail.com

Tested on an Ubuntu 24 with a 32GB VRAM Nvidia RTX 5090

Execution time: 16 min

## Setup 

1. Enable forward compatibility if your GPU and CUDA libraries are more recent and not supported by editing your matlab `startup.m` file to contain "parallel.gpu.enableCUDAForwardCompatibility(true)" and:

    ```bash
    # manually compile Kilosort3 with CUDA support for forward compatibility
    sudo apt install gcc-11 g++-11 # install gcc 11 compiler
    sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-11 100 # enable temporary
    cd /home/steeve/steeve/epfl/code/spikebias/dataset/01_intermediate/sorters/Kilosort3_buttw_forwcomp/CUDA/
    matlab -batch mexGPUall  # compile matlab mex files
    ```

2. Activate `spikesort_rtx5090` environment and select kernel

    ```bash
    python -m ipykernel install --user --name spikesort_rtx5090 --display-name "spikesort_rtx5090"
    ```

3. Run notebook or pipeline:
    ```bash
    # ks3 - npx spontaneous
    nohup python -m src.pipes.sorting.test_params.driftcorr.npx_spont.10m.ks3 \
        --recording-path dataset/00_raw/recording_npx_spont \
            --preprocess-path dataset/01_intermediate/preprocessing/recording_npx_spont \
                --sorting-path-corrected ./temp/npx_spont/SortingKS3_10m_RTX5090_DriftCorr \
                    --sorting-output-path-corrected ./temp/npx_spont/KS3_output_10m_RTX5090_DriftCorr/ \
                        --study-path-corrected ./temp/npx_spont/study_ks3_10m_RTX5090_DriftCorr/ \
                            --sorting-path-not-corrected ./temp/npx_spont/SortingKS3_10m_RTX5090_NoDriftCorr \
                                --sorting-output-path-not-corrected ./temp/npx_spont/KS3_output_10m_RTX5090_NoDriftCorr/ \
                                    --study-path-not-corrected ./temp/npx_spont/study_ks3_10m_RTX5090_NoDriftCorr/
    ```

In [7]:
%%time 
%load_ext autoreload
%autoreload 2

# import python packages
import os
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface as si
print("spikeinterface", si.__version__)

# project path
PROJ_PATH = "/home/steeve/steeve/epfl/code/spikebias/"
os.chdir(PROJ_PATH)

# import spikebias package
from src.nodes.sorting import sort_and_postprocess_10m

# recording parameters
REC_SECS = 600 
RECORDING_PATH = "./dataset/00_raw/recording_npx_spont/"

# sorting parameters
SORTER = "kilosort3"
SORTER_PATH = "/home/steeve/steeve/epfl/code/spikebias/dataset/01_intermediate/sorters/Kilosort3_buttw_forwcomp"
SORTING_OUTPUT = "./temp/SortingKS3/" 
SORTER_PARAMS = {
    "detect_threshold": 6,
    "projection_threshold": [9, 9],
    "preclust_threshold": 8,
    "car": True,
    "minFR": 0, # modified
    "minfr_goodchannels": 0, # modified
    "nblocks": 5,
    "sig": 20,
    "freq_min": 300,
    "sigmaMask": 30,
    "lam": 20.0,
    "nPCs": 3,
    "ntbuff": 64,
    "nfilt_factor": 4,
    "do_correction": True,
    "NT": 65792, # modified    
    "AUCsplit": 0.8,
    "wave_length": 61,
    "keep_good_only": False,
    "skip_kilosort_preprocessing": False,
    "scaleproc": None,
    "save_rez_to_mat": False,
    "delete_tmp_files": ("matlab_files",),
    "delete_recording_dat": False,
}

# SET KS3 software environment variable
ss.Kilosort3Sorter.set_kilosort3_path(SORTER_PATH)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
spikeinterface 0.100.5
Setting KILOSORT3_PATH environment variable for subprocess calls to: /home/steeve/steeve/epfl/code/spikebias/dataset/01_intermediate/sorters/Kilosort3_buttw_forwcomp
CPU times: user 1.52 ms, sys: 2.01 ms, total: 3.53 ms
Wall time: 14.1 ms


## npx spont w/ drift corr.

In [ ]:
# setup configuration

# WITH CORR.
CFG_CORR = {
    'probe_wiring': {
        'full': {
            'output': 'dataset/00_raw/recording_npx_spont'
        }
    },
    'preprocessing': {
        'full': {
            'output': {
                'trace_file_path': 'dataset/01_intermediate/preprocessing/recording_npx_spont'
            }
        }
    },
    'sorting': {
        'sorters': {
            f"{SORTER}": {
                '10m': {
                    'output': './temp/SortingKS3_10m_RTX5090_DriftCorr', #'path/to/sorting/output',
                    'sort_output':'./temp/KS3_output/KS3_output_10m_RTX5090_DriftCorr' #'path/to/sorting/sort_output'
                }
            }
        }
    },
    'postprocessing': {
        'waveform': {
            'sorted': {
                'study': {
                    f"{SORTER}": {  # sorter name
                        '10m': './temp/study_ks3_10m_RTX5090_DriftCorr' #'path/to/postprocessing/study'
                    }
                }
            }
        }
    }
}

# WITHOUT CORR.

CFG_NO_CORR = {
    'probe_wiring': {
        'full': {
            'output': 'dataset/00_raw/recording_npx_spont'
        }
    },
    'preprocessing': {
        'full': {
            'output': {
                'trace_file_path': 'dataset/01_intermediate/preprocessing/recording_npx_spont'
            }
        }
    },
    'sorting': {
        'sorters': {
            f"{SORTER}": {
                '10m': {
                    'output': './temp/SortingKS3_10m_RTX5090_NoDriftCorr', #'path/to/sorting/output',
                    'sort_output':'./temp/KS3_output/KS3_output_10m_RTX5090_NoDriftCorr' #'path/to/sorting/sort_output'
                }
            }
        }
    },
    'postprocessing': {
        'waveform': {
            'sorted': {
                'study': {
                    f"{SORTER}": {  # sorter name
                        '10m': './temp/study_ks3_10m_RTX5090_NoDriftCorr' #'path/to/postprocessing/study'
                    }
                }
            }
        }
    }
}

# WITH CORR.
# spike sort
sort_and_postprocess_10m(CFG_CORR, SORTER, SORTER_PARAMS, duration_sec=REC_SECS, 
                         is_sort=True, is_postpro=False, extract_wvf=False, copy_binary_recording=True,
                         remove_bad_channels=True)
# post-process
sort_and_postprocess_10m(CFG_CORR, SORTER, SORTER_PARAMS, duration_sec=REC_SECS,
                         is_sort=False, is_postpro=True, extract_wvf=True, copy_binary_recording=True,
                         remove_bad_channels=False)

# WITHOUT CORR.

SORTER_PARAMS['do_correction'] = False

# spike sort
sort_and_postprocess_10m(CFG_NO_CORR, SORTER, SORTER_PARAMS, duration_sec=REC_SECS,
                         is_sort=True, is_postpro=False, extract_wvf=False, copy_binary_recording=True,
                         remove_bad_channels=True)
# post-process
sort_and_postprocess_10m(CFG_NO_CORR, SORTER, SORTER_PARAMS, duration_sec=REC_SECS,
                         is_sort=False, is_postpro=True, extract_wvf=True, copy_binary_recording=True,
                         remove_bad_channels=False)

2025-07-10 11:05:38,567 - root - sorting.py - sort_and_postprocess_10m - INFO - Started sorting 10 minutes recording.
2025-07-10 11:05:38,571 - root - sorting.py - sort - INFO - Removing bad channels...
2025-07-10 11:05:38,573 - root - sorting.py - sort - INFO - Done removing bad channels in: 0.0
2025-07-10 11:05:38,573 - root - sorting.py - sort - INFO - Selected first 10.0 minutes in: 0.0
2025-07-10 11:05:38,574 - root - sorting.py - sort - INFO - Done converting recording as int16 in: 0.0
2025-07-10 11:05:38,574 - root - sorting.py - sort - INFO - Saving int16 binary recording...


/home/steeve/steeve/epfl/code/spikebias/envs/spikesort_rtx5090/lib/python3.9/site-packages/spikeinterface/core/base.py:1079: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.101.2 is recommended
  warnings.warn(


write_binary_recording with n_jobs = 32 and chunk_size = 400000


write_binary_recording:   0%|          | 0/60 [00:00<?, ?it/s]

2025-07-10 11:05:50,054 - root - sorting.py - sort - INFO - Done copying int16 binary recording in: 11.5
2025-07-10 11:05:50,056 - root - sorting.py - sort - INFO - Start sorting...
RUNNING SHELL SCRIPT: /home/steeve/steeve/epfl/code/spikebias/temp/KS3_output/KS3_output_10m_RTX5090_NoDriftCorr/sorter_output/run_kilosort3.sh


                            < M A T L A B (R) >

                  Copyright 1984-2024 The MathWorks, Inc.

                  R2025a (25.1.0.2943329) 64-bit (glnxa64)

                               April 16, 2025



 

To get started, type doc.

For product information, visit www.mathworks.com.

 

Steeve - forward compatibility enabled for more recent GPUs - edit startup.m file to disable

Time   0s. Computing whitening matrix.. 

Getting channel whitening matrix... 


libraries. Compilation can take several minutes. 

> In get_whitening_matrix (line 25)

In preprocessDataSub (line 62)

In kilosort3_master (line 71) 

Channel-whitening matrix computed. 

Time   

...

extract waveforms shared_memory multi buffer:   0%|          | 0/600 [00:00<?, ?it/s]

extract waveforms shared_memory multi buffer:   0%|          | 0/600 [00:00<?, ?it/s]

/home/steeve/steeve/epfl/code/spikebias/envs/spikesort_rtx5090/lib/python3.9/site-packages/spikeinterface/core/baserecordingsnippets.py:244: UserWarning: There is no Probe attached to this recording. Creating a dummy one with contact positions
  warn("There is no Probe attached to this recording. Creating a dummy one with contact positions")


extract waveforms memmap multi buffer:   0%|          | 0/600 [00:00<?, ?it/s]

2025-07-10 11:08:41,496 - root - metadata.py - set_site_and_layer - INFO - Done extracting waveforms in 24.8
2025-07-10 11:08:41,561 - root - metadata.py - set_site_and_layer - INFO - Saving layers and site contact metadata to SortingExtractor...
2025-07-10 11:08:41,580 - root - metadata.py - set_site_and_layer - INFO - Saving done in 0.0
2025-07-10 11:08:41,592 - root - sorting.py - sort_and_postprocess_10m - INFO - Done postprocessing for kilosort3- metadata written in: 25.0


In [15]:
# Compare sorting results
SortingRef = si.load_extractor("./dataset/01_intermediate/sorting/dense_probe1/SortingKS3_10m")
SortingCorr = si.load_extractor("./temp/dense_spont_probe1/SortingKS3_10m_RTX5090_DriftCorr")
SortingNoCorr = si.load_extractor("./temp/dense_spont_probe1/SortingKS3_10m_RTX5090_NoDriftCorr")

# total units
print("total units:")
print(len(SortingRef.unit_ids))
print(len(SortingCorr.unit_ids))
print(len(SortingNoCorr.unit_ids))

# total units
print("\nsingle units:")
print(sum(SortingRef.get_property('KSLabel')=='good'))
print(sum(SortingCorr.get_property('KSLabel')=='good'))
print(sum(SortingNoCorr.get_property('KSLabel')=='good'))

total units:
293
253
92

single units:
79
22
7
